# Reddit Analysis using Pushshift API

## Part 1: Data Scraping
- [Background](#Background)
- [Problem Statement](#Problem-Statement)
- [Data Scraping](#Data-Scraping)

# Background

Reddit is a website that dubs itself itself as the "Front Page of the Internet" and functions as hybrid between a news service and social media. The website is divided into subreddits, miniature communities that cover topics that range from overarching (news, science, music, etc.) to incredibly specific (AskNYC, dogecoin, siberianhusky). In these subreddits, users can make posts and up-vote or down-vote it. With enough popularity, a post can make it to the popular section of reddit where it will be seen by anyone that browses the website. This creates a self-functioning advertisement cycle that allows users to join communities they find interesting. The foundation of each post is the title, body text (contains links and images as well), and comments. In the background, users can also send and receive private messages.

In this project, we will be analyzing data from two subreddits:

1. [r/TalesFromRetail](https://www.reddit.com/r/TalesFromRetail/) (640k members): This subreddit talks about experiences from employees working in retail. The posts are mainly negative and largely refers to "Karens", people who are known for being entitled and demanding to others in a public environment.

2.  [r/raisedbynarcissists](https://www.reddit.com/r/raisedbynarcissists/) (714k members): This subreddit is a support group for people who had abusive and toxic parental figures. Posts range from personal history to questions and discussions regarding a user's personal life.

# Problem Statement
The aforementioned subreddits were chosen because both of them focus on negative experiences with specific people in their lives. The two subreddits also have a similar amount of activity. However, there is a large difference between the two as TalesFromRetail deals with one time interactions with strangers while raisedbynarcissists deals with longer and intimate history between family members.

While Karens are known for being openly obnoxious, narcissists can be soft-spoken and manipulative.This juxtaposition between public/private abusers led to a question if there were any personality similarities between the two types of people. This project has two main goals. The first is to utilize NLP methods to see what types of vocabulary can be seen in both subreddits. The second is to use classification models in order to see if the subreddit origin of a post can be found. Do Karens and narcissistic parents have any overlap?

# Data Scraping

In [1]:
import pandas as pd
import numpy as np
import requests
import time

In [3]:
# Posts for /r/raisedbynarcissists
def get_posts1():
    post_num = range(0, 1000, 50)
    base_url = 'https://api.pushshift.io/reddit/search/'

    for x in post_num:
        if x == 0:
            params = {
                'subreddit': 'raisedbynarcissists',
                'size': 50
            }
            res = requests.get(base_url + 'submission/', params = params)
            data = res.json()
            posts = pd.DataFrame(data['data'])
            earliest = posts['created_utc'].min()

            time.sleep(2)
        else:
            new_params = {
                'subreddit': 'raisedbynarcissists',
                'size': 50,
                'before': earliest
            }
            new_res = requests.get(base_url + 'submission/', params = new_params)
            new_data = new_res.json()
            new_df = pd.DataFrame(new_data['data'])
            earliest = new_df['created_utc'].min()

            posts = pd.concat([posts, new_df], ignore_index = True)

            time.sleep(2)
    df1 = posts
    df1.to_csv('../data/narcissists.csv')

In [4]:
# Posts for /r/TalesFromRetail
def get_posts2():
    post_num = range(0, 4500, 50) # 4500 posts gathered due to abundance of [removed] posts
    base_url = 'https://api.pushshift.io/reddit/search/'

    for x in post_num:
        if x == 0:
            params = {
                'subreddit': 'TalesFromRetail',
                'size': 50
            }
            res = requests.get(base_url + 'submission/', params = params)
            data = res.json()
            posts = pd.DataFrame(data['data'])
            earliest = posts['created_utc'].min()

            time.sleep(2)
        else:
            new_params = {
                'subreddit': 'TalesFromRetail',
                'size': 50,
                'before': earliest
            }
            new_res = requests.get(base_url + 'submission/', params = new_params)
            new_data = new_res.json()
            new_df = pd.DataFrame(new_data['data'])
            earliest = new_df['created_utc'].min()

            posts = pd.concat([posts, new_df], ignore_index = True)

            time.sleep(2)
    df2 = posts
    df2.to_csv('../data/retail.csv')

In [5]:
# Preliminary inspection on possibly useful columns
def insp():
    for col in df1.columns:
        try:
            if df1.drop_duplicates(subset = col).shape[0] > 100:
                print(f'Unique in {col} is {df1.drop_duplicates(subset = col).shape[0]}')
        except:
            print(f'{col} is unhashable') # TypeError: unhashable type: 'list'

    # Preliminary inspection on possibly useful columns
    for col in df2.columns:
        try:
            if df2.drop_duplicates(subset = col).shape[0] > 100:
                print(f'Unique in {col} is {df2.drop_duplicates(subset = col).shape[0]}')
        except:
            print(f'{col} is unhashable') # TypeError: unhashable type: 'list'